In [1]:
import pandas as pd
import os

In [27]:
files = ['safety-SanFrancisco.parquet','safety-Seattle.parquet','safety-Nyc.parquet']

In [2]:
directory = str(os.getcwd())+'/files/'
filename = files[0]

In [28]:
print(filename)

safety-SanFrancisco.parquet


In [3]:
df = pd.read_parquet(directory+filename)

### Data Exploration

In [4]:
# data length and data types
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3958889 entries, 0 to 3958888
Data columns (total 12 columns):
 #   Column              Dtype  
---  ------              -----  
 0   requestId           int64  
 1   dataType            object 
 2   dataSubtype         object 
 3   dateTime            object 
 4   category            object 
 5   subcategory         object 
 6   status              object 
 7   address             object 
 8   latitude            float64
 9   longitude           float64
 10  source              object 
 11  extendedProperties  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 362.4+ MB


In [5]:
df.head(2)

,requestId,dataType,dataSubtype,dateTime,category,subcategory,status,address,latitude,longitude,source,extendedProperties
0,1,Safety,911_Fire,2018-10-02 11:54:01,Potentially Life-Threatening,Medical Incident,None,ELM ST/FRANKLIN ST,37.781286,-122.422187,None,
1,2,Safety,311_All,2018-07-08 15:00:27,Street and Sidewalk Cleaning,Bulky Items,Closed,"1536 SACRAMENTO ST, SAN FRANCISCO, CA, 94109",37.791887,-122.418188,Mobile/Open311,


In [6]:
df[df.requestId==127]

,requestId,dataType,dataSubtype,dateTime,category,subcategory,status,address,latitude,longitude,source,extendedProperties
57,127,Safety,311_All,2019-02-13 18:05:00,Abandoned Vehicle,Abandoned Vehicles,Closed,"556 5TH AVE, SAN FRANCISCO, CA, 94118",37.778136,-122.462724,Web,
2968465,127,Safety,311_All,2017-12-06 23:40:00,Street and Sidewalk Cleaning,General Cleaning,Closed,"575 BERRY ST, SAN FRANCISCO, CA, 94107",37.769675,-122.400914,Mobile/Open311,


In [7]:
df[df.address == '556 5TH AVE, SAN FRANCISCO, CA, 94118']

,requestId,dataType,dataSubtype,dateTime,category,subcategory,status,address,latitude,longitude,source,extendedProperties
57,127,Safety,311_All,2019-02-13 18:05:00,Abandoned Vehicle,Abandoned Vehicles,Closed,"556 5TH AVE, SAN FRANCISCO, CA, 94118",37.778136,-122.462724,Web,
16032,38806,Safety,311_All,2016-10-18 18:28:05,Sidewalk or Curb,Curb_or_Curb_Ramp_Defect,Closed,"556 5TH AVE, SAN FRANCISCO, CA, 94118",37.778131,-122.462845,Phone,
507322,31368,Safety,311_All,2019-01-26 15:10:00,Parking Enforcement,Blocking_Driveway_Cite_Only,Closed,"556 5TH AVE, SAN FRANCISCO, CA, 94118",37.778136,-122.462724,Web,
668372,422252,Safety,311_All,2018-07-03 16:34:00,Parking Enforcement,Blocking_Driveway_Cite_Only,Closed,"556 5TH AVE, SAN FRANCISCO, CA, 94118",37.778132,-122.462842,Phone,
914502,1019767,Safety,311_All,2017-05-02 13:58:52,Graffiti,Graffiti on Other_enter_additional_details_below,Closed,"556 5TH AVE, SAN FRANCISCO, CA, 94118",37.778135,-122.462734,Web,
1856771,905280,Safety,311_All,2019-08-23 09:44:42,Encampments,Encampment items,Closed,"556 5TH AVE, SAN FRANCISCO, CA, 94118",37.778125,-122.462766,Mobile/Open311,
2432657,1102385,Safety,311_All,2019-01-25 19:35:00,Parking Enforcement,Blocking_Driveway_Cite_Only,Closed,"556 5TH AVE, SAN FRANCISCO, CA, 94118",37.778136,-122.462724,Web,
3272968,739475,Safety,311_All,2018-09-22 20:18:00,Parking Enforcement,Blocking_Driveway_Cite_Only,Closed,"556 5TH AVE, SAN FRANCISCO, CA, 94118",37.778132,-122.462842,Phone,
3587037,298489,Safety,311_All,2019-11-14 11:44:00,Parking Enforcement,Blocking_Driveway_Cite_Only,Closed,"556 5TH AVE, SAN FRANCISCO, CA, 94118",37.778132,-122.462842,Phone,


In [8]:
# measure the number of nulls
df.isnull().sum()

requestId                   0
dataType                    0
dataSubtype                 0
dateTime                    0
category                 4933
subcategory                 0
status                1405089
address                     3
latitude                    0
longitude                   0
source                1405089
extendedProperties          0
dtype: int64

In [9]:
# based on the unique value count, we can safely remove dataType and extendedProperties from the dataframe
df.drop(columns=['dataType','extendedProperties'], axis=1, inplace=True)

In [10]:
# convert dateTime column values to datetime
df["dateTime"]= pd.to_datetime(df["dateTime"]) 

In [11]:
# categorize columns into numeric/datetime columns and string/object columns
numeric_col = ['requestId','dateTime','latitude','longitude']
string_col = ['dataSubtype', 'category','subcategory', 'status', 'address','source']

In [12]:
# measuring how many unique values are in each column to understand which can be categorized
for col in df[string_col].columns.values:
    print(col+ ': ' + str(len(df[col].unique())))

dataSubtype: 2
category: 96
subcategory: 639
status: 3
address: 226725
source: 9


In [13]:
# some columns cannot be categorized, add category list
category_col = ['dataSubtype','category','status','source']

In [14]:
# set category for the columns identified in category_col
# for cat in category_col:
#     df[cat] = df[cat].astype('category')
df[category_col] = df[category_col].astype('category')

In [15]:
# check data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3958889 entries, 0 to 3958888
Data columns (total 10 columns):
 #   Column       Dtype         
---  ------       -----         
 0   requestId    int64         
 1   dataSubtype  category      
 2   dateTime     datetime64[ns]
 3   category     category      
 4   subcategory  object        
 5   status       category      
 6   address      object        
 7   latitude     float64       
 8   longitude    float64       
 9   source       category      
dtypes: category(4), datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 196.3+ MB


In [16]:
# check for nulls 
df.isnull().sum()

requestId            0
dataSubtype          0
dateTime             0
category          4933
subcategory          0
status         1405089
address              3
latitude             0
longitude            0
source         1405089
dtype: int64

In [17]:
# aggregating on the category with the fewest types, we can see 911 has most of the nulls
df.groupby(['dataSubtype']).count()

,requestId,dateTime,category,subcategory,status,address,latitude,longitude,source
dataSubtype,,,,,,,,,
311_All,2553800,2553800,2553800,2553800,2553800,2553797,2553800,2553800,2553800
911_Fire,1405089,1405089,1400156,1405089,0,1405089,1405089,1405089,0


In [18]:
#understand the unique categories of source and status 
print(df.source.unique())
print(df.status.unique())

[NaN, Mobile/Open311, Phone, Web, Integrated Agency, Other Department, Twitter, Email, Mail]
Categories (8, object): [Mobile/Open311, Phone, Web, Integrated Agency, Other Department, Twitter, Email, Mail]
[NaN, Closed, Open]
Categories (2, object): [Closed, Open]


In [19]:
#assume all 911 calls are via phone and closed due to visitation (if required)
df.loc[(df['dataSubtype']=='911_Fire') & pd.isna(df['source']), 'source'] = 'Phone'
df.loc[(df['dataSubtype']=='911_Fire') & pd.isna(df['status']), 'status'] = 'Closed'

In [21]:
# check for nulls, leave the rest alone - could remove since minor group compared to total
df.isnull().sum()

requestId         0
dataSubtype       0
dateTime          0
category       4933
subcategory       0
status            0
address           3
latitude          0
longitude         0
source            0
dtype: int64

In [22]:
# check for duplicates
df.duplicated().sum()

1

In [23]:
# drop duplicates 
df.drop_duplicates(keep = 'first', inplace = True)

In [24]:
# check duplicate removal worked
df.duplicated().sum()

0

In [25]:
df = df.reset_index(drop=True)

In [26]:
# write to file
df.to_parquet(directory+'cleaned_safety-SanFrancisco.parquet', compression='snappy') 

### Write function with some of the above steps:
- convert dateTime column to_datetime
- remove columns dataType, extendedProperties
- for 911 calls, replace null status and source with 'Closed' and 'Phone'
- remove duplicates
- reset index
- write to file

In [55]:
def data_preprocessing(directory, filename):
    """Preprocess dataframe
    - convert dateTime column to_datetime
    - remove columns dataType, extendedProperties
    - for 911 calls, replace null status and source with 'Closed' and 'Phone'
    - remove duplicates
    - reset index
    - write to file
    """
    filepath = directory + filename
   
    df = pd.read_parquet(filepath)
    print("\n\nProcessing: " + filename)
    
    df.drop(columns=['dataType','extendedProperties'], axis=1, inplace=True)
    print("Dropping columns: dataType, extendedProperties")

    df["dateTime"]= pd.to_datetime(df["dateTime"])
    print("Converted dateTime column to datetime")

    df.loc[(df['dataSubtype']=='911_Fire') & pd.isna(df['source']), 'source'] = 'Phone'
    df.loc[(df['dataSubtype']=='911_Fire') & pd.isna(df['status']), 'status'] = 'Closed'
    print("Converted 911_Fire dataSubtypes with source/status null to Phone and Closed")
    
    df[['dataSubtype','category','status','source']] = df[['dataSubtype','category','status','source']].astype('category')
    print("Converted to category these columns: dataSubtype,category,status,source")    
    
    print('Removing ' + str(df.duplicated().sum()) + ' duplicates')
    df.drop_duplicates(keep = 'first', inplace = True)

    df = df.reset_index(drop=True)
    print("Resetting index")
    
    output_file = 'cleaned_'+str(filename) 
    df.to_parquet(directory+output_file, compression='snappy')
    print("Writing cleaned file to directory: " + output_file )
    
    return print('Processing of ' + str(filename) +' complete')
    


In [56]:
for i in files:
    data_preprocessing(directory, i)



Processing: safety-SanFrancisco.parquet
Dropping columns: dataType, extendedProperties
Converted dateTime column to datetime
Converted 911_Fire dataSubtypes with source/status null to Phone and Closed
Converted to category these columns: dataSubtype,category,status,source
Removing 1 duplicates
Resetting index
Writing cleaned file to directory: cleaned_safety-SanFrancisco.parquet
Processing of safety-SanFrancisco.parquet complete


Processing: safety-Seattle.parquet
Dropping columns: dataType, extendedProperties
Converted dateTime column to datetime
Converted 911_Fire dataSubtypes with source/status null to Phone and Closed
Converted to category these columns: dataSubtype,category,status,source
Removing 0 duplicates
Resetting index
Writing cleaned file to directory: cleaned_safety-Seattle.parquet
Processing of safety-Seattle.parquet complete


Processing: safety-Nyc.parquet
Dropping columns: dataType, extendedProperties
Converted dateTime column to datetime
Converted 911_Fire dataSubt